# Introduction

Our business problem began by identifying the most common products bought together and for that we did several recommendation systems.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('orders.csv')
df.drop(columns="Unnamed: 0", axis =1, inplace=True)
df.head()

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2
2,1001096,6001131,2023-01-17,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3
3,1121833,6358142,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,12.0,Não,AMB,Bebidas,Aguas Minerais,4
4,1122758,6328488,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,60.0,Não,AMB,Bebidas,Aguas Minerais,5


How the company works is that for every deivery place in a day a truck takes the products, there can be more than one delivery place per day, as well several clients can have the same delivery place

In [2]:
#Implementing a orders Id
#order = df.groupby(['date','delivery_place']).size().reset_index()
#order['order_id'] = [i+1 for i in range(len(order))]
#order = order.drop(0,axis=1)
#data=df.merge(order, on=['date','delivery_place'])
#data.head()

In [3]:
#make groupby to get indexes product_tyoe and subtype
analise = df.groupby(['warehouse_zone','product_type','product_subtype'])['quantity'].sum().reset_index()
analise2 = df.groupby(['warehouse_zone','product_type',])['quantity'].sum().reset_index()

#Create Id's 
analise['cellule'] = (analise.groupby('product_type').cumcount()+1).apply(lambda x: str(x).zfill(2))
analise2['ID'] = (analise2.groupby('warehouse_zone').cumcount()+1).apply(lambda x: str(x).zfill(2))
analise2['alley']= analise2.apply(lambda row: row['warehouse_zone'] +'-'+ str(row['ID']), axis=1)

#Drop unecessary columns
analise2=analise2.drop(['quantity','ID'], axis=1)
analise=analise.drop(['quantity'],axis=1)

#Merge everything together on a dataframe
df_locations = df.merge(analise2, on=['warehouse_zone','product_type'], how='left')
df_new = df_locations.merge(analise, on=['warehouse_zone','product_type','product_subtype'], how='left')
df_new['alleycell'] = df_new.apply(lambda row: row['alley'] +'-'+ str(row['cellule']), axis=1)
df_new.head(3)

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id,alley,cellule,alleycell
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1,AMB-01,02,AMB-01-02
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2,AMB-01,02,AMB-01-02
2,1001096,6001131,2023-01-17,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3,AMB-01,02,AMB-01-02


In [18]:
coords = pd.read_csv("coords.csv", delimiter=';')
coords['x'] = coords['x'].str.replace(',', '.').astype(float)
coords['y'] = coords['y'].str.replace(',', '.').astype(float)

gg = df_new.groupby(['warehouse_zone','product_type','alley']).size().reset_index()
s= coords.merge(gg, on=['warehouse_zone','product_type'], how='left')
s= s.drop(0,axis=1)
s.loc[len(s)] = ['START',"START", 15, 1,"START"]
s.tail()

,warehouse_zone,product_type,x,y,alley
51,REF,Peixe,5.5,34.0,REF-09
52,REF,Refeições Cook & Chill (Socigeste),5.5,32.0,REF-10
53,SAL,Legumes,27.0,25.5,SAL-01
54,SAL,Mercearia,27.0,20.5,SAL-02
55,START,START,15.0,1.0,START


## Recommendation system

## Apriori

### Association Rules

With the order id in place we can now start building a recommendation system based on what items are frequently purchased together. This can be useful for suggesting related items to customers, as well as informing product placement and marketing strategies.

To do this, we use the Apriori algorithm to identify frequent itemsets - that is, sets of items that appear together in a minimum number of orders. From these itemsets, we can generate association rules that tell us which items tend to be purchased together.

The Apriori algorithm is a scalable and efficient way to mine large datasets for frequent itemsets and generate association rules. By using this approach, we can identify patterns and relationships in transactional data that can inform our recommendation system and ultimately help us make data-driven decisions about product recommendations, marketing strategies, and product placement.

In [5]:
from mlxtend.frequent_patterns import apriori, association_rules

# Filter for orders with more than one item
df = df.groupby('order_id').filter(lambda x: len(x) > 1)

# Pivot data to create binary matrix
basket = pd.pivot_table(df, index='order_id', columns='product_description', values='quantity', aggfunc='sum', fill_value=0)

# Convert values to binary
basket[basket > 0] = 1

# Find frequent itemsets
freq_itemsets = apriori(basket, min_support=0.05, use_colnames=True)


freq_itemsets.head(5)

C:\Users\berna\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.062928,(ABOBORA)
1,0.135336,(ACUCAR GRANULADO SACO PAPEL 1 KG RAR)
2,0.058423,"(AGUA SPRING PORTUGAL 1,5L PET)"
3,0.231300,(ALFACE)
4,0.072407,(ALHOS SECOS)


In the context of the Apriori algorithm, support refers to the frequency with which an itemset appears in the dataset. Specifically, the support of an itemset is defined as the proportion of transactions in the dataset that contain all the items in that itemset.

For example, in the table you provided, the first itemset (`ABOBORA`) has a support value of `0.062928`. This means that about `6.3%` of the transactions in the dataset contain the item `ABOBORA`. Similarly, the second itemset (`ACUCAR GRANULADO SACO PAPEL 1 KG RAR`) has a support value of `0.135336`, which means that about `13.5%` of the transactions in the dataset contain the item `ACUCAR GRANULADO SACO PAPEL 1 KG RAR`.

The support metric is important in the Apriori algorithm because it is used to identify frequent itemsets, which are then used to generate association rules. Specifically, itemsets with a support value above a given threshold (e.g., `0.05` or `0.1`) are considered frequent, and the algorithm generates association rules based on those itemsets.

In [6]:
# Generate association rules
rules = association_rules(freq_itemsets, metric='lift', min_threshold=1)

# Sort rules by lift and support
rules = rules.sort_values(['lift', 'support'], ascending=[False, False])

# Print top 5 rules
print(rules.shape)
rules.head(5)

(972, 10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
925,"(ARROZ LONGO COMUM AGULHA EUROCHEFE 1 KG, CENO...","(ALFACE, BATATA BRANCA)",0.071516,0.139700,0.054247,0.758530,5.429721,0.044256,3.562765,0.878667
928,"(ALFACE, BATATA BRANCA)","(ARROZ LONGO COMUM AGULHA EUROCHEFE 1 KG, CENO...",0.139700,0.071516,0.054247,0.388310,5.429721,0.044256,1.517901,0.948307
918,"(ALFACE, CENOURA, ARROZ LONGO COMUM AGULHA EUR...","(BATATA BRANCA, TOMATE MEDIO)",0.085875,0.117879,0.054247,0.631694,5.358837,0.044124,2.395076,0.889804
935,"(BATATA BRANCA, TOMATE MEDIO)","(ALFACE, CENOURA, ARROZ LONGO COMUM AGULHA EUR...",0.117879,0.085875,0.054247,0.460191,5.358837,0.044124,1.693423,0.922087
950,"(ALFACE, CENOURA, COUVE LOMBARDO)","(BATATA BRANCA, TOMATE MEDIO)",0.097325,0.117879,0.060863,0.625362,5.305118,0.049391,2.354593,0.898998


- Metrics:
    - antecedents:
        - the antecedent itemset of the association rule
    - consequents: 
        - the consequent itemset of the association rule
    - antecedent support: 
        - the support of the antecedent itemset, i.e., the proportion of transactions that contain all the items in the antecedent
    - consequent support: 
        - the support of the consequent itemset, i.e., the proportion of transactions that contain all the items in the consequent
    - support:
        - the support of the rule, i.e., the proportion of transactions that contain both the antecedent and the consequent
    - confidence: 
        - the confidence of the rule, i.e., the proportion of transactions that contain the consequent given that they also contain the antecedent
    - lift: 
        - the lift of the rule, which measures how much more often the antecedent and consequent co-occur in the dataset than we would expect if they were independent. A lift greater than 1 indicates a positive correlation between the antecedent and consequent, while a lift less than 1 indicates a negative correlation, and a lift equal to 1 indicates independence.
    - leverage: 
        - the leverage of the rule, which measures the difference between the observed frequency of the antecedent and consequent co-occurring and the frequency that would be expected if they were independent. A leverage of 0 indicates independence, while a positive leverage indicates a positive correlation, and a negative leverage indicates a negative correlation.
    - conviction: 
        - the conviction of the rule, which measures how much the antecedent and consequent are dependent on each other. A conviction value greater than 1 indicates that the antecedent and consequent are positively dependent, while a conviction value less than 1 indicates that they are negatively dependent, and a conviction value equal to 1 indicates independence.
    - zhang's metric: 
        - a metric that combines lift and conviction to give an overall measure of the interestingness of the rule. A value greater than 0 indicates that the rule is interesting.
        
The most important metrics for evaluating association rules are usually **support, confidence, and lift**. Support and confidence are used to filter out rules that are not frequent or not strong enough, while lift is used to identify rules that represent interesting correlations between items. In general, a high support value indicates that the rule is frequent, a high confidence value indicates that the rule is strong, and a high lift value indicates that the rule is interesting.

In [7]:
rules[(rules['confidence']>=0.7) & (rules['support']>= 0.05)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
925,"(ARROZ LONGO COMUM AGULHA EUROCHEFE 1 KG, CENO...","(ALFACE, BATATA BRANCA)",0.071516,0.139700,0.054247,0.758530,5.429721,0.044256,3.562765,0.878667
948,"(ALFACE, BATATA BRANCA, COUVE LOMBARDO)","(CENOURA, TOMATE MEDIO)",0.079212,0.146176,0.060863,0.768365,5.256455,0.049285,3.686076,0.879418
847,"(ALFACE, PEPINO)","(CENOURA, TOMATE MEDIO)",0.078508,0.146176,0.059690,0.760311,5.201356,0.048214,3.562215,0.876559
956,"(CENOURA, COUVE LOMBARDO, TOMATE MEDIO)","(ALFACE, BATATA BRANCA)",0.083904,0.139700,0.060863,0.725391,5.192507,0.049142,3.132825,0.881365
598,"(ARROZ LONGO COMUM AGULHA EUROCHEFE 1 KG, CENO...","(ALFACE, BATATA BRANCA)",0.092539,0.139700,0.066776,0.721602,5.165384,0.053849,3.090186,0.888637
...,...,...,...,...,...,...,...,...,...,...
45,(PERNA FRANGO N/CALIBRADA EXTRA CONG.),(ALFACE),0.096058,0.231300,0.077194,0.803615,3.474343,0.054976,3.914252,0.787856
129,(PERNA FRANGO N/CALIBRADA EXTRA CONG.),(CENOURA),0.096058,0.207367,0.068653,0.714704,3.446561,0.048734,2.778286,0.785289
11,(ARROZ LONGO COMUM AGULHA EUROCHEFE 1 KG),(ALFACE),0.139090,0.231300,0.108494,0.780027,3.372363,0.076322,3.494520,0.817126
49,(PURE DE BATATA EM FLOCOS LUTOSA 1 KG),(ALFACE),0.081746,0.231300,0.060582,0.741102,3.204075,0.041674,2.969126,0.749136


Explicar

## Q-Learn

In [12]:
test = df_new[df_new["order_id"] == 8]

In [19]:
import numpy as np
import pandas as pd
import time
import math

start_time = time.time()

# Function for calculating distance between zones
def calculate_distance(zone1, zone2):
    x1, y1 = float(s[s["alley"] == zone1]["x"]), float(s[s["alley"] == zone1]["y"]) # dots[zone1][0], dots[zone1][1]
    x2, y2 = float(s[s["alley"] == zone2]["x"]), float(s[s["alley"] == zone2]["y"])
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def get_shortest_distance_zone(zone_list):
    distances = []
    for zone in zone_list:
        distances.append(calculate_distance("START", zone))
    return zone_list[np.argmin(distances)]

# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
n_episodes = 10 #1000  # Number of episodes

# Get the unique order ids and zones
order_ids = test['order_id'].unique()
zones = test['alley'].unique()
min_order_id = np.min(order_ids)

# Initialize the Q-table
num_zones = len(order_ids)
q_table = np.zeros((len(order_ids), len(zones), len(zones)))

for episode in range(n_episodes):
    for order_id in order_ids:
        order = test[test['order_id'] == order_id].sort_values(by='date')
        zone_list = order['alley'].tolist()

        for i in range(len(zone_list) - 1):
            current_zone = zone_list[i]
            next_zone = zone_list[i + 1]

            current_zone_idx = np.where(zones == current_zone)[0][0]
            next_zone_idx = np.where(zones == next_zone)[0][0]

            # Choose action (next zone) using epsilon-greedy strategy
            if np.random.random() < epsilon:
                action_idx = np.random.randint(num_zones)
            else:
                action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

            # Calculate the reward based on the distance to the chosen zone
            reward = -calculate_distance(current_zone, zones[min(action_idx, len(zones)-1)])

            # Update Q-table
            q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] = q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] + alpha * (reward + gamma * np.max(q_table[order_id - min_order_id, min(action_idx, len(zones)-1), :]) - q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)])

# Print the optimal picking order
for order_id in order_ids:
    order = test[test['order_id'] == order_id].sort_values(by='date')
    zone_list = order['alley'].tolist()

    first_zone = get_shortest_distance_zone(zone_list)
    optimal_order = [first_zone]

    for i in range(len(zone_list) - 1):
        current_zone = zone_list[i + 1]
        current_zone_idx = np.where(zones == current_zone)[0][0]

        # Choose the best action based on the Q-table
        action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

        if zones[action_idx] not in optimal_order:
            optimal_order.append(zones[action_idx])

    print(f"Optimal picking order for order {order_id}: {optimal_order}")

end_time = time.time()

print("Running time:", end_time - start_time, "seconds")

Optimal picking order for order 8: ['FRL-02', 'NAL-06', 'CON-09', 'AMB-10', 'AMB-01', 'CON-03', 'REF-08', 'CON-07', 'FRL-01', 'CON-04', 'REF-03', 'AMB-12', 'AMB-07', 'AMB-04', 'CON-06', 'AMB-13']
Running time: 2.707690715789795 seconds
